## ACP sur le dataset des médicaments

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
PandasTools.RenderImagesInAllDataFrames(images=True)
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules
from IPython import display
import matplotlib.pyplot as plt 
import prince   #bibliothèque équivalente à FactoMineR


### On commence par ouvrir le dataset

In [ ]:
df_meds = PandasTools.LoadSDF('', isomericSmiles=True)
df_meds.head()

### Préparation

In [ ]:
#On regarde d'abord ce qu'on a, notamment les types de molécules
print(len(df_meds))

In [ ]:
#Avec la fonction value counts on regarde ce qu'on a dans la colonne 'molecule_type'


In [ ]:
#Pour regarder une catégorie précise avec un filtre
df_meds[df_meds['molecule_type'] == 'Oligosaccharide'].head()

In [ ]:
#On s'intéresse uniquement au Small molecule, filtrer pour ne garder que cette catégorie


In [ ]:
len(df_meds)

In [ ]:
#Ensuite, ajoute une colonne permettant de voire si la molécule est en mélange ou seule
def number_frags(mol):
    #Get the differents fragments
    
    #Get the number
    
    #return the number


df_meds['Mixture'] = df_meds['ROMol'].apply(number_frags)
df_meds.head()

In [ ]:
#On peut regarder la distribution avec la fonction value counts

In [ ]:
#On garde uniquement les molécules seules
df_meds = 
len(df_meds)

In [ ]:
#On peut aussi regarder les dates d'approbation des médicaments
df_meds['first_approval'] = df_meds['first_approval'].astype(float) #On transforme la colonne en float
df_meds['first_approval'].value_counts()

In [ ]:
#On supprime les dates non précisées
df_meds.dropna(subset=['first_approval'], inplace=True)
len(df_meds)

In [ ]:
#Et pour finir on garde que les médicaments sortis après les années 2000


In [ ]:
%matplotlib inline

#On peut regarder sous forme d'histogramme
df_meds['first_approval'].hist()


In [ ]:
#On prépare ce qu'on a besoin pour l'ACP : 'ID', 'natural_product', 'ROMol'
df_PCA = df_meds[['ID', 'natural_product', 'ROMol']]
df_PCA.head()

### Calcul des descripteurs

In [ ]:
#On importe les modules nécessaires 
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Lipinski

#### Liste des descripteurs : 
#### ['FCSP3', 'HBA', 'HBD', 'LabuteASA', 'LogP', 'MQN10', 'MQN8', 'MW', 'NAR', 'NRB', 'TPSA']

In [ ]:
df_PCA['MW'] = df_PCA.ROMol.map(lambda m: round(Chem.Descriptors.ExactMolWt(m), 1))
df_PCA['LogP'] = df_PCA.ROMol.map(lambda m: round(Chem.Descriptors.MolLogP(m), 1))
df_PCA['TPSA'] = df_PCA.ROMol.map(lambda m: round(Chem.Descriptors.TPSA(m), 1))
df_PCA['LabuteASA'] = df_PCA.ROMol.map(lambda m: round(Chem.Descriptors.LabuteASA(m), 1))
df_PCA['HBA'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.NumHAcceptors(m))
df_PCA['HBD'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.NumHDonors(m))
df_PCA['FCSP3'] = df_PCA.ROMol.map(lambda m: Lipinski.FractionCSP3(m))
df_PCA['MQN8'] = df_PCA.ROMol.map(lambda m: rdMolDescriptors.MQNs_(m)[7])
df_PCA['MQN10'] = df_PCA.ROMol.map(lambda m: rdMolDescriptors.MQNs_(m)[9])
df_PCA['NAR'] = df_PCA.ROMol.map(lambda m: Lipinski.NumAromaticRings(m))
df_PCA['NRB'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.NumRotatableBonds(m))

In [ ]:
df_PCA.head()

### Visualisation des descripteurs

In [ ]:
#Distribution des MWs avec la fonction hist


In [ ]:
#On garde en dessous de 700 et au dessus de 200 avec le filtre between


In [ ]:
#on peut vérifier avec un histogramme


In [ ]:
#On peut regarder la projection des molécules selon différentes variables avec plot
df_PCA[['MW', 'LogP']].plot()

In [ ]:
#Il faut préciser le type de plot
df_PCA.plot(x='MW', y='LogP', kind='scatter')

In [ ]:
#Si on veut colorier les points selon le type de molécule
ax = df_PCA[df_PCA['natural_product'] == '0'].plot.scatter(x='MW', y='LogP', color='DarkBlue', label='Non Natural')
df_PCA[df_PCA['natural_product'] == '1'].plot.scatter(x='MW', y='LogP', color='DarkGreen', label='Natural', ax=ax)

In [ ]:
#On peut regarder les valeurs des différents descripteurs avec la fonction plot.box


In [ ]:
#Sans prendre en compte les valeurs trop grandes
x = list(range(7,13))
x.append(4)
df_PCA.iloc[:, x].plot.box()

In [ ]:
#La librairie seaborn peut-être plus adaptée pour les graphiques 
sns.distplot(df_PCA.MW, )

In [ ]:
#On peut regarder la distribution sur le même graphique selon la catégorie
sns.distplot(df_PCA[df_PCA['natural_product'] == '0']['MW'] , color="skyblue", label="Non Natural", kde=False)
sns.distplot(df_PCA[df_PCA['natural_product'] == '1']['MW'] , color="green", label="Natural", kde=False)
plt.legend()

In [ ]:
#Pour regarder tous les descripteurs :
plt.figure(figsize=(20, 16))

j=1
for i in df_PCA.iloc[:, 3:].columns:
    plt.subplot(4, 4, j)
    sns.distplot(df_PCA[df_PCA['natural_product'] == '0'][i] , color="skyblue", label="Non Natural", kde=False)
    sns.distplot(df_PCA[df_PCA['natural_product'] == '1'][i] , color="green", label="Natural", kde=False)
    plt.legend()
    j += 1


In [ ]:
#On peut obtenir une matrice de corrélation avec la fonction corr


In [ ]:
#Montrer visuellement les deux variables les mieux correlées


### Début de l'ACP

On va utiliser la bibliothèque prince, une bibliothèque facile à utiliser, il faut donc commencer par l'installer dans votre environnement
    - Nouvau terminal
    - Activation de votre terminal
    - pip install prince=='0.2.6'  (on installe une ancienne version)


In [ ]:
#utilisation de Prince par un exemple classique 

import matplotlib.pyplot as plt
import prince


df = pd.read_csv('https://raw.githubusercontent.com/kormilitzin/Prince/master/examples/data/iris.csv')
#on lit un datasetr tout fait 

pca = prince.PCA(df, n_components=4)

fig1, ax1 = pca.plot_cumulative_inertia()
fig2, ax2 = pca.plot_correlation_circle()
fig3, ax3 = pca.plot_rows(axes=[0,1], color_by='class', ellipse_fill=True)

plt.show()

## On reprend notre dataframe de médicaments

In [ ]:
#On transforme la colonne natural product en type "category


In [ ]:
#On commence par faire une copie de notre dataframe car Prince va effectuer des changements dessus
df_PCA2 = df_PCA.copy()

In [ ]:
#Pour lancer l'ACP on applique la fonction PCA à notre DF
pca1 = prince.PCA(df_PCA2, n_components=4)

In [ ]:
#L'attribut X nous donne les valeurs standardisées
pca1.X.head()

In [ ]:
#Les fonctions pour représenter l'ACP sont incluses dans le package. 
#On peut commencer par montrer l'inertie cumulative


In [ ]:
#Le cercle des corrélations


In [ ]:
#Enfin, les variables selon les principales composantes
fig3, ax3 = pca1.plot_rows(axes=[0,1], color_by='natural_product', ellipse_fill=True)
plt.show()

### Analyse

In [ ]:
#pour avoir les valeurs standardisées des descripteurs 
#Attribut X de pca1


In [ ]:
#Pour avoir les corrélations des variables dans les composantes
pca1.column_correlations

In [ ]:
import numpy as np
#Pour obtenir le cos2 des variables 
cos2 = np.square(pca1.column_correlations)
cos2

In [ ]:
#Pour obtenir les contributions de chaque variable, il faut diviser le cos2 par la somme totale des cos2
contrib  = (cos2*100) / cos2.sum()
contrib

In [ ]:
#Contribution de chaque individu dans les composantes

In [ ]:
#On peut aussi représenter les autres composantes


In [ ]:
#Il est aussi possible de représenter en 3D

#Besoin des indexes pour chaque catégorie
non_nat = pca1.categorical_columns[pca1.categorical_columns.natural_product == '0'].index
nat = pca1.categorical_columns[pca1.categorical_columns.natural_product == '1'].index


%pylab inline
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams['legend.fontsize'] = 10   
ax.plot(pca1.row_principal_coordinates[0][non_nat], pca1.row_principal_coordinates[1][non_nat], pca1.row_principal_coordinates[2][non_nat], 'o', markersize=8, color='red', alpha=0.5, label='Non-Natural')
ax.plot(pca1.row_principal_coordinates[0][nat], pca1.row_principal_coordinates[1][nat], pca1.row_principal_coordinates[2][nat], '^', markersize=8, alpha=0.5, color='green', label='Natural')

plt.title('3D PCA Projection for drugs approved since 2000')
ax.legend(loc='lower right')

plt.show()

### Essayons avec moins de descripteurs mais plus séparatifs

In [ ]:
df_PCA_new = df_PCA[['ID', 'natural_product',]]
df_PCA2 = df_PCA_new.copy()
pca1 = prince.PCA(df_PCA2, n_components=4)

fig1, ax1 = pca1.plot_cumulative_inertia()
fig2, ax2 = pca1.plot_correlation_circle()
fig3, ax3 = pca1.plot_rows(axes=[0,1], color_by='natural_product', ellipse_fill=True)


plt.show()

### Peut-on séparer les méthodes d'administrations?

In [ ]:
df_meds.head()

In [ ]:
#on précise pour chaque médicament la méthode d'administration
#On ne veut pas de mix

Ladm=[]
for a, b in df_meds[['oral', 'parenteral', 'topical']].iterrows():
    oral = b[0]
    parenteral = b[1]
    topical = b[2]
    if oral == 'True' and parenteral == 'False' and topical == 'False' :#que oral
        Ladm.append('O')
    elif parenteral == 'True' and oral == 'False' and topical == 'False' : #que parenteral
        Ladm.append('P')
    elif topical == 'True' and parenteral == 'False' and oral == 'False': #que topical
        Ladm.append('T')
    else : #mélange
        Ladm.append('M')


len(Ladm)

In [ ]:
df_meds['Administration'] = Ladm #on crée la nouvelle colonne

#Filtre pour prendre uniquement les non mixtes

df_meds.head()

In [ ]:
#On recalcule des descripteurs

df_PCA = df_meds[['ROMol', 'Administration']]
# df_PCA['MW'] = df_PCA.ROMol.map(lambda m: round(Chem.Descriptors.ExactMolWt(m), 1))
# df_PCA['LogP'] = df_PCA.ROMol.map(lambda m: round(Chem.Descriptors.MolLogP(m), 1))
# df_PCA['TPSA'] = df_PCA.ROMol.map(lambda m: round(Chem.Descriptors.TPSA(m), 1))
# df_PCA['HBA'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.NumHAcceptors(m))
# df_PCA['HBD'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.NumHDonors(m))
# df_PCA['FCSP3'] = df_PCA.ROMol.map(lambda m: Lipinski.FractionCSP3(m))
# df_PCA['MQN10'] = df_PCA.ROMol.map(lambda m: rdMolDescriptors.MQNs_(m)[9])
# df_PCA['NAR'] = df_PCA.ROMol.map(lambda m: Lipinski.NumAromaticRings(m))
# df_PCA['NRB'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.NumRotatableBonds(m))
# df_PCA['Chi0'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.Chi0(m))
# df_PCA['Bertz'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.BertzCT(m))
# df_PCA['ES'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.Chi1(m))
# df_PCA['FP'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.Kappa1(m))
# df_PCA['FP1'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.Kappa2(m))
# df_PCA['FP2'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.Kappa3(m))

df_PCA['FCSP3'] = df_PCA.ROMol.map(lambda m: Lipinski.FractionCSP3(m))
df_PCA['NAR'] = df_PCA.ROMol.map(lambda m: Lipinski.NumAromaticRings(m))
df_PCA['Chi0'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.Chi0(m))
df_PCA['LogP'] = df_PCA.ROMol.map(lambda m: round(Chem.Descriptors.MolLogP(m), 1))
df_PCA['TPSA'] = df_PCA.ROMol.map(lambda m: round(Chem.Descriptors.TPSA(m), 1))
df_PCA['HBA'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.NumHAcceptors(m))
df_PCA['Bertz'] = df_PCA.ROMol.map(lambda m: Chem.Descriptors.BertzCT(m))
len(df_PCA)

In [ ]:
#ACP

df_PCA2=df_PCA.copy()
pca1 = prince.PCA(df_PCA2, n_components=4)

fig1, ax1 = pca1.plot_cumulative_inertia()
fig2, ax2 = pca1.plot_rows(color_by='Administration')

plt.show()

### ACP sur un autre dataset 

In [ ]:
#Charger le dataset mix.sdf 
df_mix = PandasTools.LoadSDF('', 
                             isomericSmiles=True)
df_mix.head()

In [ ]:
#Calculer les descripteurs pour chaque molécule 
df_mix['MW'] = df_mix.ROMol.map(lambda m: round(Chem.Descriptors.ExactMolWt(m), 1))
df_mix['LogP'] = df_mix.ROMol.map(lambda m: round(Chem.Descriptors.MolLogP(m), 1))
df_mix['TPSA'] = df_mix.ROMol.map(lambda m: round(Chem.Descriptors.TPSA(m), 1))
df_mix['LabuteASA'] = df_mix.ROMol.map(lambda m: round(Chem.Descriptors.LabuteASA(m), 1))
df_mix['HBA'] = df_mix.ROMol.map(lambda m: Chem.Descriptors.NumHAcceptors(m))
df_mix['HBD'] = df_mix.ROMol.map(lambda m: Chem.Descriptors.NumHDonors(m))
df_mix['FCSP3'] = df_mix.ROMol.map(lambda m: Lipinski.FractionCSP3(m))
df_mix['MQN8'] = df_mix.ROMol.map(lambda m: rdMolDescriptors.MQNs_(m)[7])
df_mix['MQN10'] = df_mix.ROMol.map(lambda m: rdMolDescriptors.MQNs_(m)[9])
df_mix['NAR'] = df_mix.ROMol.map(lambda m: Lipinski.NumAromaticRings(m))
df_mix['NRB'] = df_mix.ROMol.map(lambda m: Chem.Descriptors.NumRotatableBonds(m))

In [ ]:
df_mix.head()

In [ ]:
#LA colonne catégorie s'appelle cat
#Réaliser l'ACP avec 4 composantes et montrer l'inertie cumulée et la projection

### Avec Scikit-Learn

In [ ]:
#Il faut commencer par distinguer les variables descriptives des variables catégoriques
X = df_PCA.iloc[:, 3:].values  #descriptives
y = df_PCA.iloc[:, 1].values   #catégoriques


In [ ]:
#Il faut ensuite normaliser les données 
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)   #On instantie la classe PCA de scikit learn
pca.fit(X_std) #On l'applique sur nos données
X_reduced = pca.transform(X_std) #On récupère les résultats
print("Reduced dataset shape:", X_reduced.shape)


In [ ]:
#Pour la projection selon l'axe 1 et 2

def color(x):
    if x == '0' :
        return('red')
    else :
        return'green'


import pylab as pl
pl.scatter(X_reduced[:, 0], X_reduced[:, 1], c=[color(x) for x in df_PCA.natural_product],
           cmap='RdYlBu')



In [ ]:
#Coordonnées des variables
coords = pd.DataFrame(pca.components_.T * np.sqrt(pca.explained_variance_))
coords

In [ ]:
#Le cos2
cos2 = np.square(coords)
cos2

In [ ]:
#La contribution
contrib  = (cos2*100) / cos2.sum()
contrib

In [ ]:
#Pour la variance cumulative
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
#Pour créer un biplot 
#Attention à adapter en fonction de vos données

#Exemple avec dataset iris
import pandas as pd

df = pd.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
    header=None,
    sep=',')

df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
df.dropna(how="all", inplace=True) # drops the empty line at file-end
# split data table into data X and class labels y

X = df.ix[:,0:4].values
y = df.ix[:,4].values

from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)




from matplotlib.patches import Circle

pca = PCA(n_components=2).fit(X_std)
reduced_data = pca.transform(X_std)
pca_samples = pca.transform(X_std)
reduced_data = pd.DataFrame(reduced_data, columns = ['Dimension 1', 'Dimension 2'])

def biplot(data, reduced_data, pca):
    
    fig, ax = plt.subplots(figsize = (14,12))
    circ = plt.Circle((0,0),radius = 1, ec='b', fc='none')
    ax.add_patch(circ)
     # projections of the original features
    feature_vectors = np.array(coords[[0,1]])

    for i, v in enumerate(feature_vectors):
        print(v)
        ax.scatter(v[0], v[1])

        ax.arrow(0, 0, v[0], v[1], head_width=0.02, head_length=0.01, linewidth=0.2, color='red')
        ax.text(v[0], v[1], df.ix[:,0:4].columns[i], color='black', ha='center', va='center', fontsize=18)
    # scatterplot of the reduced data 
    ax.scatter(x=reduced_data.loc[:, 'Dimension 1'], y=reduced_data.loc[:, 'Dimension 2'], facecolors='b', edgecolors='b', s=70, alpha=0.5)
    


   

    ax.set_xlabel("Dimension 1", fontsize=14)
    ax.set_ylabel("Dimension 2", fontsize=14)
    ax.set_title("PC plane with original feature projections.", fontsize=16);
    return ax

biplot(X_std, reduced_data, pca)

In [ ]:
#autre méthode 
pca = PCA(n_components=4)
pca.fit(X_std)
T = pca.transform(X_std)

import math

def get_important_features(transformed_features, components_, columns):
    """
    This function will return the most "important" 
    features so we can determine which have the most
    effect on multi-dimensional scaling
    """
    num_columns = len(columns)

    # Scale the principal components by the max value in
    # the transformed set belonging to that component
    xvector = components_[0] * max(transformed_features[:,0])
    yvector = components_[1] * max(transformed_features[:,1])

    # Sort each column by it's length. These are your *original*
    # columns, not the principal components.
    important_features = { columns[i] : math.sqrt(xvector[i]**2 + yvector[i]**2) for i in range(num_columns) }
    important_features = sorted(zip(important_features.values(), important_features.keys()), reverse=True)
    print("Features by importance:\n", important_features)

get_important_features(T, pca.components_, df.ix[:,0:4].columns)

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

def draw_vectors(transformed_features, components_, columns):
    """
    This funtion will project your *original* features
    onto your principal component feature-space, so that you can
    visualize how "important" each one was in the
    multi-dimensional scaling
    """

    num_columns = len(columns)

    # Scale the principal components by the max value in
    # the transformed set belonging to that component
    xvector = components_[0] * max(transformed_features[:,0])
    yvector = components_[1] * max(transformed_features[:,1])

    ax = plt.axes()

    for i in range(num_columns):
    # Use an arrow to project each original feature as a
    # labeled vector on your principal component axes
        plt.arrow(0, 0, xvector[i], yvector[i], color='b', width=0.0005, head_width=0.02, alpha=0.75)
        plt.text(xvector[i]*1.2, yvector[i]*1.2, list(columns)[i], color='b', alpha=0.75)

    return ax

def color(x):
    if x == 'Iris-setosa' :
        return('red')
    elif x == 'Iris-versicolor':
        return('blue')
    else :
        return'green'

ax = draw_vectors(T, pca.components_, df.ix[:,0:4].columns.values)
T_df = pd.DataFrame(T)
T_df.columns = ['component1', 'component2', 'component3', 'component4']

plt.xlabel('Principle Component 1')
plt.ylabel('Principle Component 2')
plt.scatter(T_df['component1'], T_df['component2'], color=[color(x) for x in df['class']], alpha=0.5)
plt.show()